In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install timm segmentation_models_pytorch rasterio torchmetrics transformers
!apt install -y zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [4]:
params = {
    'backbone': 'segformer_b1',
    'weights': 'noisy-student',
    'learning_rate': 0.001,
    'epochs': 30,
    'batch_size': 24,
    'num_workers': 12,
    'device': torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'),
    'div_factor':10,
    'final_div_factor':100,
    'threshhold':0.5
}

## Functions

We define a function on which we can measure the accuracy of our model.

In [5]:
def sort_paths_by_number(paths):
    return sorted(paths, key=lambda x: int(x.split('_')[-1].split('.tif')[0]))

In [6]:
def calculate_f1_score(pred, true, threshold=0.5):
    """
    Compute the F1 score, also known as balanced F-score or F-measure.

    Parameters:
    - true: ground truth (correct) labels, a tensor of shape [N, H, W].
    - pred: predicted labels, a tensor of shape [N, 1, H, W].
    - threshold: the prediction threshold, defaults to 0.5.

    Returns:
    - F1 score
    """

    # Flatten the tensors
    true = true.numpy().flatten()
    pred = pred.numpy().flatten()

    # Calculate F1 score
    f1 = f1_score(true, pred)

    return f1

In [7]:
def intersection_and_union(pred, true):
    """
    Calculates intersection and union for a batch of images.

    Args:
        pred (torch.Tensor): a tensor of predictions
        true (torc.Tensor): a tensor of labels

    Returns:
        intersection (int): total intersection of pixels
        union (int): total union of pixels
    """
    pred = pred.squeeze(1)
    valid_pixel_mask = true.ne(255)  # valid pixel mask
    true = true.masked_select(valid_pixel_mask)
    pred = pred.masked_select(valid_pixel_mask)

    # Convert to CPU and NumPy for logical operations
    true = true.numpy()
    pred = pred.numpy()

    # Intersection and union totals
    intersection = np.logical_and(true, pred)
    union = np.logical_or(true, pred)
    return intersection.sum(), union.sum()

In [8]:
def save_model(epoch,model,ckpt_path='./',name='segformer',val_iou=0):
    path = os.path.join(ckpt_path, '{}_wo_ca.pth'.format(name))
    torch.save(model.state_dict(), path, _use_new_zipfile_serialization=False)

def load_model(model,ckpt_path):
    state = torch.load(ckpt_path)
    model.load_state_dict(state)
    return model

## Loss Functions

In [9]:
class FocalLoss(torch.nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights
        self.bce = torch.nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, input, target):
        #ce_loss = F.binary_cross_entropy_with_logits(input.squeeze(1), target.long(),reduction=self.reduction,weight=self.weight)
        ce_loss = self.bce(input.squeeze(1), target.squeeze(1))
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [10]:
class XEDiceLoss(torch.nn.Module):
    """
    Computes (0.5 * CrossEntropyLoss) + (0.5 * DiceLoss).
    """

    def __init__(self):
        super().__init__()
        self.xe = torch.nn.BCEWithLogitsLoss(reduction="none")
        #self.xe = FocalLoss(reduction='none')

    def forward(self, pred, true):

        pred = pred.squeeze(1)
        valid_pixel_mask = true.ne(255)  # valid pixel mask

        # Cross-entropy loss
        temp_true = torch.where((true == 255), 0, true)  # cast 255 to 0 temporarily
        xe_loss = self.xe(pred, temp_true.float())
        xe_loss = xe_loss.masked_select(valid_pixel_mask).mean()

        # Dice loss

        pred = pred.sigmoid() #torch.softmax(pred, dim=1)[:, 1]

        pred = pred.masked_select(valid_pixel_mask)
        true = true.masked_select(valid_pixel_mask)

        dice_loss = 1 - (2.0 * torch.sum(pred * true) + 1e-7) / (torch.sum(pred + true) + 1e-7)

        #print(xe_loss,dice_loss,(2.0 * torch.sum(pred * true) + 1e-7))

        return (0.5 * xe_loss) + (0.5 * dice_loss)

## Data
We create the Dataset and Dataloader

In [11]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split

import glob
import pandas as pd
import tifffile
import os

In [12]:
PATH_SRC = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/src'
PATH_OUTPUT = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/submit'
PATHS_VAL = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/evaluation_true_color/evaluation_true_color_*.tif'))
PATHS_VAL_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/sample/evaluation_mask_*.tif'))
PATHS_TRAIN = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_*.tif'))
PATHS_TRAIN_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_*.tif'))

In [13]:
class DataTransform():
    def __init__(self):
        self.data_transform = {
            "train": A.Compose(
                [
                  A.ShiftScaleRotate(shift_limit=0.0625,rotate_limit=15,p=0.5),
                  A.GridDistortion(p=0.35),
                  A.HorizontalFlip(p=0.5),
                  A.VerticalFlip(p=0.5),
                  A.GaussianBlur(p=0.25),
                  # A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  A.Resize(512, 512, interpolation=1, p=1),
                  ToTensorV2()
                ]
            ),
            "val": A.Compose(
                [
                  # A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  A.Resize(512, 512, interpolation=1, p=1),
                  ToTensorV2()
                ]
            )
        }

    def __call__(self, phase, img, mask):
      return self.data_transform[phase](image=img, mask=mask)

In [14]:
class CloudDataset(Dataset):
  def __init__(self, df, phase, transform):
    self.data = df
    self.phase = phase
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def normalize_data(self, image_tiff):
    # Normalize the image for better visualization
    for i in range(3):  # Assuming the image has 3 channels
        image_tiff[:,:,i] = (image_tiff[:,:,i] - image_tiff[:,:,i].min()) / (image_tiff[:,:,i].max() - image_tiff[:,:,i].min())

    return image_tiff

  def __getitem__(self, index):
    row = self.data.iloc[index]
    true_color_PATH = row['true_color']
    mask_PATH = row['mask']
    img = tifffile.imread(true_color_PATH)
    img = self.normalize_data(img.astype(np.float32))
    mask = tifffile.imread(mask_PATH)
    mask = mask.astype(np.float32)

    img_and_mask = self.transform(self.phase, img, mask)
    img = img_and_mask["image"]
    mask = img_and_mask["mask"]

    return img, mask

In [15]:
df = pd.DataFrame({
    'true_color': PATHS_TRAIN,
    'mask': PATHS_TRAIN_MASK
})

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
print(df.head())

df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = CloudDataset(df_train, phase="train", transform=DataTransform())
train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)


val_dataset = CloudDataset(df_val, phase="val", transform=DataTransform())
val_dataloader = DataLoader(
    val_dataset,
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)

dataloaders_dict = {
    'train': train_dataloader,
    'val': val_dataloader
}

                                                                                        true_color                                                                                 mask
0  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_0.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_0.tif
1  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_1.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_1.tif
2  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_2.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_2.tif
3  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_3.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_3.tif
4  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/trai

<ipython-input-15-44df3b6a699d>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Model
We implement the cloud detection Model based on the Microsoft Competion Winner: https://github.com/drivendataorg/cloud-cover-winners/blob/main/1st%20place/src/unet_efficient_net_b1_train_fp16.py

In [16]:
from transformers import SegformerForSemanticSegmentation
import torch.nn as nn

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.segformer = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b1-finetuned-ade-512-512')
        self.segformer.decode_head.classifier = nn.Conv2d(256,1,kernel_size=1)
    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        image = image
        mask = self.segformer(image).logits
        mask = F.interpolate(mask, image.shape[-2:], mode="bilinear", align_corners=True)
        mask = mask.squeeze(1)

        return mask


In [19]:
model = Net()
model.to(params['device'])

Net(
  (segformer): SegformerForSemanticSegmentation(
    (segformer): SegformerModel(
      (encoder): SegformerEncoder(
        (patch_embeddings): ModuleList(
          (0): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
            (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          )
          (1): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          )
          (2): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
          )
          (3): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (layer_norm

## Training

In [20]:
from tqdm import tqdm
import matplotlib.pyplot as plt

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

scheduler = torch.optim.lr_scheduler.OneCycleLR(
      optimizer,
      steps_per_epoch = 42,
      pct_start = 0.1,
      max_lr = params['learning_rate'],
      epochs = params['epochs'],
      div_factor = params['div_factor'],
      final_div_factor = params['final_div_factor'],
      verbose = True
    )

# criterion = FocalLoss()
criterion = XEDiceLoss()

Adjusting learning rate of group 0 to 1.0000e-04.


In [22]:
def train(model, img, mask):
  model.train()
  optimizer.zero_grad()

  preds = model(img)
  loss = criterion(preds, mask)
  # loss = criterion(preds.squeeze(dim=1), mask.squeeze(dim=1))
  loss.backward()
  optimizer.step()
  loss = loss.item()

  # Calculate validation IOU (global)
  preds = (preds.detach().sigmoid().cpu() > params['threshhold']) *1
  mask = mask.detach().cpu()

  # plt.imshow(preds[0][0], cmap='gray')
  # plt.colorbar()
  # plt.title("pred")
  # plt.show()

  # plt.imshow(mask[0], cmap='gray')
  # plt.colorbar()
  # plt.title("mask")
  # plt.show()


  intersection, union = intersection_and_union(preds, mask)
  f1_score = calculate_f1_score(preds, mask)

  return loss, intersection, union, f1_score

In [23]:
def val(model, img, mask):
  model.eval()
  with torch.no_grad():
    preds = model(img)
    loss = criterion(preds, mask)
    # loss = criterion(preds.squeeze(dim=1), mask.squeeze(dim=1))
    loss = loss.item()

    preds = (preds.detach().sigmoid().cpu() > params['threshhold']) *1 #torch.softmax(preds, dim=1)[:, 1]
    mask = mask.detach().cpu()
    intersection, union = intersection_and_union(preds, mask)
    f1_score = calculate_f1_score(preds, mask)

  return loss, intersection, union, f1_score

Delete GPU-RAM

In [ ]:
import gc  # Garbage Collector
gc.collect()
torch.cuda.empty_cache()

In [24]:
best_iou = 0

for epoch in range(1, params['epochs']+1):
    print(f'\nEpoch: {epoch}')
    for phase in ['train', 'val']:
        total_intersection=0
        total_union=0
        total_loss=0
        total_f1=0

        pbar = tqdm(enumerate(dataloaders_dict[phase]),
                              total=len(dataloaders_dict[phase]),
                              leave=True,
                              dynamic_ncols=True)
        for i, (img, mask) in pbar:
            img, mask = img.to(params['device'], non_blocking=True), mask.to(params['device'], non_blocking=True)

            # training
            if phase == 'train':
                loss, intersection, union, f1 = train(model, img, mask)
            # validation
            else:
                loss, intersection, union, f1 = val(model, img, mask)

            # free GPU
            del img
            del mask

            total_intersection += intersection
            total_union += union
            total_loss += loss
            iou = total_intersection/total_union
            total_f1 += f1
            pbar.set_postfix({'iou': iou, 'loss': total_loss/(i+1), 'f1': total_f1/(i+1)})

        total_loss /= len(dataloaders_dict[phase])
        total_f1 /= len(dataloaders_dict[phase])

        if phase == 'train':
          scheduler.step()

        print(f'    Phase: {phase} -->  loss={total_loss} iou={iou} f1={total_f1}')

    # save best model
    if iou > best_iou:
      print(f'Saving model with iou {iou}')
      save_model(epoch, model, ckpt_path=PATH_SRC, val_iou=iou)
      best_iou = iou
    else:
      print(f'Not saving for iou {iou}')




Epoch: 1


100%|██████████| 34/34 [03:09<00:00,  5.56s/it, iou=0.501, loss=0.458, f1=0.695]

Adjusting learning rate of group 0 to 1.0014e-04.
    Phase: train -->  loss=0.45755885541439056 iou=0.5009078315136963 f1=0.6947050914645714



100%|██████████| 9/9 [01:01<00:00,  6.84s/it, iou=0.68, loss=0.346, f1=0.8]


    Phase: val -->  loss=0.34601373473803204 iou=0.6797498490780662 f1=0.8004121998997387
Saving model with iou 0.6797498490780662

Epoch: 2


...

    Phase: val -->  loss=0.1399662411875195 iou=0.7805940836030402 f1=0.8777199873862127
Not saving for iou 0.7805940836030402

Epoch: 30


100%|██████████| 34/34 [02:26<00:00,  4.30s/it, iou=0.818, loss=0.117, f1=0.896]


Adjusting learning rate of group 0 to 2.2196e-04.
    Phase: train -->  loss=0.11683815191773807 iou=0.8180899070598731 f1=0.8961679688067773


100%|██████████| 9/9 [00:35<00:00,  3.97s/it, iou=0.778, loss=0.168, f1=0.847]

    Phase: val -->  loss=0.16827560795678032 iou=0.7776023095196631 f1=0.8473229725558498
Not saving for iou 0.7776023095196631
